
---



---




<div style="text-align: center; font-size: 30px; font-weight: bold;">
  TIMELAPSE ANUAL (DIG - IGN)
</div>



---



---

In [ ]:
import os
import ee
import geemap
import ipywidgets as widgets
from IPython.display import display

import urllib


from PIL import Image, ImageSequence

import calendar
from tqdm import tqdm

from ipywidgets import Button, Dropdown, VBox, Layout

In [7]:
ee.Initialize()# Inicializar Earth Engine

Map = geemap.Map(center=(-40, -64), zoom=4)
Map.add_basemap('HYBRID') #Ejemplos de mapas base: ROADMAP, SATELLITE, TERRAIN, HYBRID
Map

Map(center=[-40, -64], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(ch…

# Landsat

- 'Red/Green/Blue': Color verdadero
- 'NIR/Red/Green': Color natural
- 'SWIR2/SWIR1/NIR': Falso color de vegetación
- 'NIR/SWIR1/Red': Color de vegetación
- 'SWIR2/NIR/Red': Índice de vegetación de diferencia normalizada (NDVI)
- 'SWIR2/SWIR1/Red': Índice de agua (NDWI)
- 'SWIR1/NIR/Blue': Índice de suelo (Soil Index)
- 'NIR/SWIR1/Blue': Índice de humedad del suelo (Soil Moisture Index)
- 'SWIR2/NIR/Green': Índice de vegetación de ajuste de suelo (SAVI)
- 'SWIR1/NIR/Red': Índice de vegetación mejorado (EVI)


In [8]:

#..............................................................................# Boton
# Evento de clic del botón
def submit_clicked(b):
  with output:
    output.clear_output()
    if start_year.value >= end_year.value:
      print('El año inicial debe ser menor que el año final.')
      return
    print('Procesando...')


    timelapse_L = geemap.landsat_timelapse(
      roi=Map.user_roi,
      out_gif='landsat.gif',
      start_year=start_year.value,
      end_year=end_year.value,
      start_date='01-01',
      end_date='12-31',
      frequency='year',
      bands=bands.value.split('/'),
      frames_per_second=speed.value,
      title=title.value,
      font_color=font_color.value,
      font_size=font_size.value,
      progress_bar_color=progress_bar_color.value,
      #mp4=True,
      apply_fmask=True,
      #fading=True, #desvanecimiento
    )

  # Muestra el timelapse en el mapa
    geemap.show_image(timelapse_L)

#..............................................................................#


In [9]:
def download_timelapse(b):
    # Descarga el archivo en el dispositivo local
    files.download('landsat.gif')

In [10]:
#..............................................................................# Interfaz grafica
style = {'description_width': 'initial'}

title = widgets.Text(
    description='Título:', value='Landsat Timelapse - DIG - IGN', width=200, style=style
)

bands = widgets.Dropdown(
    description='Seleccionar combinación de bandas:',
    options=[
        'Red/Green/Blue',
        'NIR/Red/Green',
        'SWIR2/SWIR1/NIR',
        'NIR/SWIR1/Red',
        'SWIR2/NIR/Red',
        'SWIR2/SWIR1/Red',
        'SWIR1/NIR/Blue',
        'NIR/SWIR1/Blue',
        'SWIR2/NIR/Green',
        'SWIR1/NIR/Red',
    ],
    value='Red/Green/Blue',
    style=style,
    layout=widgets.Layout(width='30%')  # Ajusta el ancho del widget
)
hbox1 = widgets.HBox([title, bands])

#-------------------------------------------------------------------------------#
start_year = widgets.IntSlider(
    description='Año inicial:', value=1984, min=1984, max=2023, style=style
)

end_year = widgets.IntSlider(
    description='Año final:', value=2023, min=1984, max=2023, style=style
)
hbox2 = widgets.HBox([start_year, end_year])

#-------------------------------------------------------------------------------#
speed = widgets.IntSlider(
  description='Frames por segundo:',
  tooltip='Frames por segundo:',
  value=4,
  min=1,
  max=30,
  style=style,
)
hbox3 = widgets.HBox([speed])

#-------------------------------------------------------------------------------#
font_size = widgets.IntSlider(
    description='Tamaño de fuente:', value=30, min=10, max=50, style=style
)

font_color = widgets.ColorPicker(
    concise=False, description='Color de fuente:', value='white', style=style
)

progress_bar_color = widgets.ColorPicker(
    concise=False, description='Color de la barra de progreso:', value='blue', style=style
)

hbox4 = widgets.HBox([font_size, font_color, progress_bar_color])

#-------------------------------------------------------------------------------#
# Crea la etiqueta
label = widgets.Label("") ## ?

#-------------------------------------------------------------------------------#

#----------  Botón  ----------#
# Establece el ancho del botón
button_width = '200px'  # Puedes ajustar el valor según tus necesidades

submit = widgets.Button(
    description='Crear timelapse con Landsat', ##ver
    button_style='primary',
    tooltip='Click en el botón para crear el timelapse con imágenes Landsat',
    layout=widgets.Layout(width=button_width)
)

#----------  Botón descarga ----------#
boton_descarga = widgets.Button(
    description='Descarga', ##ver
    button_style='primary',
    tooltip='Click en el botón para descargar',
    layout=widgets.Layout(width=button_width)
)


#-------------------------------------------------------------------------------#
output = widgets.Output() #

#si se presiona el botón se llama a la función
submit.on_click(submit_clicked)

boton_descarga.on_click(download_timelapse)

#-------------------------------------------------------------------------------#
etiqueta_linea = widgets.Label(value='____________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________')


# Parámetros L

In [17]:
widgets.VBox([etiqueta_linea, hbox1, label, hbox2,label, hbox3, hbox4,label, submit,boton_descarga, etiqueta_linea]) #imprimir interfaz

In [18]:
output #ejecutar esta sentencia nuevamente, de ser necesario, para imprimir resultado. Tener en cuenta que puede tardar, dependiendo de la extención del área de interés.

Output()

# Sentinel 2

- 'Red', 'Green', 'Blue': Color verdadero
- 'NIR', 'Red', 'Green': Índice de vegetación de diferencia normalizada (NDVI)
- 'SWIR2', 'SWIR1', 'Red': Índice de Diferencia Normalizada Edificada (NDBI)
- 'NIR', 'SWIR1', 'Red': Índice de agua (NDWI)
- 'NIR/Red/Blue': Índice de vegetación mejorado (EVI)

In [11]:
#..............................................................................#
# Evento de clic del botón
def submit_clicked_s2(b):
  with output_s:
    output_s.clear_output()
    if start_year_s2.value >= end_year_s2.value:
      print('El año inicial debe ser menor que el año final.')
      return
    print('Computing...')



    timelapse_S2 = geemap.sentinel2_timelapse(
      roi=Map.user_roi,
      out_gif='sentinel2.gif', ###### este parametro se puede modificar. Tener en cuenta que no debe haber espacios.
      start_year=start_year_s2.value,
      end_year=end_year_s2.value,
      start_date='01-01',
      end_date='12-31',
      frequency='year',
      #cloud_pct=60,
      bands=bands_s2.value.split('/'),
      frames_per_second=speed_s2.value,
      title=title_s2.value,
      font_color=font_color_s.value,
      font_size=font_size_s.value,
      progress_bar_color=progress_bar_color_s.value,
      mp4=True,
      apply_fmask=False,
      #fading=True, #desvanecimiento
    )

  # Muestra el timelapse en el mapa
    geemap.show_image(timelapse_S2)
    #Map.add_layer(timelapse)




#..............................................................................#
def download_timelapse_s2(b):
  pass


In [12]:
style = {'description_width': 'initial'}
title_s2 = widgets.Text(
    description='Título:', value='Sentinel-2 Timelapse - DIG - IGN', width=200, style=style
)

bands_s2 = widgets.Dropdown(
    description='Seleccionar combinación de bandas:',
    options=[
          'Red/Green/Blue',
          'NIR/Red/Green',
          'SWIR2/SWIR1/Red',
          'NIR/SWIR1/Red',
          'NIR/Red/Blue',
    ],
    value='Red/Green/Blue',
    style=style,
    layout=widgets.Layout(width='30%')  # Ajusta el ancho del widget
)
hbox1_s2 = widgets.HBox([title_s2, bands_s2])

#-------------------------------------------------------------------------------#
start_year_s2 = widgets.IntSlider(
    description='Año inicial:', value=2015, min=2015, max=2023, style=style
)

end_year_s2 = widgets.IntSlider(
    description='Año final:', value=2023, min=2015, max=2023, style=style
)
hbox2_s2 = widgets.HBox([start_year_s2, end_year_s2])

#-------------------------------------------------------------------------------#

speed_s2 = widgets.IntSlider(
  description='Frames por segundo:',
  tooltip='Frames por segundo:',
  value=4,
  min=1,
  max=10,
  style=style,
)
hbox3_s2 = widgets.HBox([speed_s2])

#-------------------------------------------------------------------------------#
font_size_s = widgets.IntSlider(
    description='Tamaño de fuente:', value=30, min=10, max=50, style=style
)

font_color_s = widgets.ColorPicker(
    concise=False, description='Color de fuente:', value='white', style=style
)

progress_bar_color_s = widgets.ColorPicker(
    concise=False, description='Color de la barra de progreso:', value='blue', style=style
)

hbox4_s2 = widgets.HBox([font_size_s, font_color_s, progress_bar_color_s])

#-------------------------------------------------------------------------------#


#-------------------------------------------------------------------------------#

# Crea la etiqueta
label = widgets.Label("")


#----------  Botón  ----------#
# Establece el ancho del botón
button_width = '200px'  # Puedes ajustar el valor según tus necesidades

submit_s2 = widgets.Button(
    description='Crear timelapse con Sentinel-2',
    button_style='primary',
    tooltip='Click en el botón para crear el timelapse en el mapa',
    layout=widgets.Layout(width=button_width)
)

output_s = widgets.Output()

#----------  Botón  descarga ----------#
boton_descarga_s2 = widgets.Button(
    description='descargar timelapse con Sentinel-2',
    button_style='primary',
    tooltip='descarga',
    layout=widgets.Layout(width=button_width)
)








#si se presiona el botón se llama a la función
submit_s2.on_click(submit_clicked_s2)

boton_descarga_s2.on_click(download_timelapse_s2)

# Parámetros s2

In [19]:
widgets.VBox([etiqueta_linea, hbox1_s2, label, hbox2_s2,label, hbox3_s2, hbox4_s2, label, submit_s2, boton_descarga_s2, etiqueta_linea]) #imprimir interfaz

In [ ]:
output_s